In [19]:
from jupyter_plotly_dash import JupyterDash
from IPython import display
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import plotly.express as px


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Modelf
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "jacob"
password = "fortress"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.readAll())
print(df)
dx = px.data.tips()



                            _id     age_upon_outcome animal_id animal_type  \
0      60f085b7f31f75b369bccc31   2           1 year   A725717         Cat   
1      60f085b7f31f75b369bccc32   4         7 months   A733653         Cat   
2      60f085b7f31f75b369bccc33   1          3 years   A746874         Cat   
3      60f085b7f31f75b369bccc34  10         3 months   A664290         Cat   
4      60f085b7f31f75b369bccc35  11           1 year   A721199         Dog   
...                         ...  ..              ...       ...         ...   
10044  611083871f0e3c5d24df71ce   2           2 year   A725720         Dog   
10045  611083911f0e3c5d24df71d0   2           2 year   A725720         Dog   
10046  6110839e1f0e3c5d24df71d2   2           2 year   A725720         Dog   
10047  611083b31f0e3c5d24df71d4   2           2 year   A725720         Dog   
10048  611083f31f0e3c5d24df71d6   2           2 year   A725720         Dog   

                        breed         color date_of_birth  \
0 

In [20]:
#########################
# Dashboard Layout / View
#########################
app = dash.Dash(__name__)
app = JupyterDash('AnimalCollection')


app.layout = html.Div([
    
    html.Img(src='logo.png'),
    html.Div(className = 'row',
            style = {'display': 'flex'},
            children = [
                html.Button(id='submit-button-one', n_clicks=0,children='Water'),
                html.Button(id='submit-button-two', n_clicks=0,children='Mountain'),
                html.Button(id='submit-button-three', n_clicks=0,children='Disaster'),
                html.Button(id='submit-button-four', n_clicks=0,children='Reset')
            ]),
    
    html.Div(id='hidden-div', style={'display':'none'}),

    html.Center(html.B(html.H1('Grazioso Salvare'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #data = df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,

    ),
    html.Br(),
     html.Hr(),
     html.Div(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )


])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output("pie-chart","figure"),
    [Input("breed","value"),
    Input("values", "value")])
def generate_chart(breed, values):
    fig = px.pie(dx, values=values, breed=breed)
    return fig



@app.callback(
    Output('datatable-id',"data"),
    [Input('submit-button-one','n_clicks'),Input('submit-button-two','n_clicks'),Input('submit-button-three','n_clicks'),Input('submit-button-four','n_clicks')
                ])

def on_click(bt1,bt2,bt3,bt4):
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.readAll())
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.readAll({
            "breed": "Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland",
            "sex_upon_outcome" : "Intact Female",
            "age_upon_outcome_in_weeks":{$gt: 26, $lt:126}
        })))
    elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.readAll({
            "breed": "German Shepherd", "Alaskan Malamute", "Old English Sheepdog","Siberian Husky","Rottweiler",
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome_in_weeks":{$gt: 26, $lt:126}
            
        })))
    elif (int(bt3) > int(bt2)):
        df = pd.DataFrame(list(shelter.readAll({
            "breed": "Doberman Pinscher", "German Shepherd", "Golden Retriever","Bloodhound","Rottweiler",
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome_in_weeks":{$gt: 20, $lt:300}
        })))
    elif (int(bt4) > int(bt3)):
        df = pd.DataFrame(list(shelter.readAll()))
        
    return df.to_dict('records')
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




In [21]:
app